In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = saw
band = [2]
commitnumber = de5d43e
molecule = {'h2o': 'atmpro'}
tsfc = 257


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band2
  atmpro = mls
  band = 2
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 0
  molecule = h2o
  ng_adju = [-2, -2, 0]
  ng_refs = [2, 3, 6]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 540
  vmin = 340
  w_diffuse = [(1.66, 1.66), (1.8, 1.8, 1.8), (1.8, 1.66, 1.45, 1.45, 1.45, 1.45)]
  wgt = [(0.6, 0.6), (0.6, 0.6, 0.6), (0.6, 0.6, 0.6, 0.6, 0.6, 0.6)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.126104                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.067545
0.000750      2.0  0.135432                  2.0  0.074382
0.001052      3.0  0.159255                  3.0  0.093661
0.001476      4.0  0.184836                  4.0  0.113236
0.002070      5.0  0.212358                  5.0  0.134415
0.002904      6.0  0.241026                  6.0  0.157194
0.004074      7.0  0.270651                  7.0  0.182732
0.005714      8.0  0.300452                  8.0  0.210669
0.008015      9.0  0.329539                  9.0  0.239965
0.011243     10.0  0.356778                 10.0  0.270035
0.015771     11.0  0.380712                 11.0  0.300193
0.022122     12.0  0.399897                 12.0  0.329415
0.031031     13.0  0.412769                 13.0  0.355897
0.043528     14.0  0.412162                 14.0  0.372194
0.061057     15.0  0.392992                 15.0  0.371237
0.085645     16.0  0.363204                 16.0  0.356143
0.120136     17.0  0.330525                 17.0  0.331173
0.168516     18.0  0.297977                 18.0  0.298946
0.236378     19.0  0.268807                 19.0  0.265453
0.331549     20.0  0.246717                 20.0  0.239722
0.465100     21.0  0.235437                 21.0  0.230375
0.652400     22.0  0.202341                 22.0  0.205718
0.915100     23.0  0.146597                 23.0  0.157063
1.283650     24.0  0.105410                 24.0  0.117234
1.800600     25.0  0.080712                 25.0  0.089520
2.525700     26.0  0.064747                 26.0  0.068812
3.542800     27.0  0.053107                 27.0  0.053834
4.969550     28.0  0.046117                 28.0  0.045159
6.970850     29.0  0.043784                 29.0  0.042528
9.778100     30.0  0.042211                 30.0  0.040141
13.715850    31.0  0.039664                 31.0  0.036405
19.239350    32.0  0.036538                 32.0  0.033130
26.987250    33.0  0.036171                 33.0  0.034004
37.855300    34.0  0.038999                 34.0  0.037243
53.100050    35.0  0.041637                 35.0  0.039171
73.887500    36.0  0.043847                 36.0  0.041283
97.662500    37.0  0.045387                 37.0  0.044233
121.437500   38.0  0.045595                 38.0  0.046274
145.212500   39.0  0.059984                 39.0  0.062773
168.987500   40.0  0.072645                 40.0  0.076953
192.762500   41.0  0.084191                 41.0  0.088070
216.537500   42.0  0.088756                 42.0  0.090416
240.312500   43.0  0.091147                 43.0  0.089712
264.087500   44.0  0.092560                 44.0  0.087947
287.862500   45.0  0.091337                 45.0  0.083631
311.637500   46.0  0.098337                 46.0  0.087924
335.412500   47.0  0.123935                 47.0  0.112271
359.187500   48.0  0.158493                 48.0  0.146111
382.962500   49.0  0.237489                 49.0  0.220573
406.737500   50.0  0.343994                 50.0  0.319709
430.512500   51.0  0.441217                 51.0  0.404650
454.287500   52.0  0.520795                 52.0  0.466465
478.062500   53.0  0.581304                 53.0  0.522324
501.837500   54.0  0.627116                 54.0  0.582736
525.612500   55.0  0.663064                 55.0  0.633835
549.387500   56.0  0.690957                 56.0  0.663444
573.162500   57.0  0.710582                 57.0  0.680840
596.937500   58.0  0.720252                 58.0  0.696428
620.712500   59.0  0.719377                 59.0  0.710084
644.487500   60.0  0.708396                 60.0  0.715672
668.262500   61.0  0.688736                 61.0  0.706836
692.037500   62.0  0.662415                 62.0  0.683350
715.812500   63.0  0.631507                 63.0  0.650102
739.587500   64.0  0.598349                 64.0  0.613358
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -51.937341   0.000000 -51.937341        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -51.863529   0.000003   
6.244000e-04 2     -51.937349   0.000018 -51.937331 -51.863537   0.000015   
8.759000e-04 3     -51.937353   0.000026 -51.937327 -51.863540   0.000020   
1.228600e-03 4     -51.937358   0.000037 -51.937321 -51.863544   0.000029   
1.723400e-03 5     -51.937364   0.000054 -51.937310 -51.863550   0.000041   
2.417400e-03 6     -51.937373   0.000080 -51.937292 -51.863557   0.000060   
3.390900e-03 7     -51.937384   0.000120 -51.937265 -51.863568   0.000088   
4.756500e-03 8     -51.937399   0.000179 -51.937221 -51.863581   0.000131   
6.672000e-03 9     -51.937419   0.000266 -51.937153 -51.863598   0.000196   
9.358900e-03 10    -51.937444   0.000396 -51.937048 -51.863620   0.000294   
1.312780e-02 11    -51.937474   0.000586 -51.936889 -51.863648   0.000442   
1.841450e-02 12    -51.937511   0.000861 -51.936650 -51.863680   0.000663   
2.583020e-02 13    -51.937553   0.001254 -51.936299 -51.863717   0.000989   
3.623230e-02 14    -51.937597   0.001807 -51.935790 -51.863755   0.001466   
5.082340e-02 15    -51.937639   0.002561 -51.935078 -51.863790   0.002144   
7.129060e-02 16    -51.937676   0.003551 -51.934125 -51.863819   0.003073   
1.000000e-01 17    -51.937705   0.004815 -51.932890 -51.863836   0.004302   
1.402710e-01 18    -51.937709   0.006396 -51.931313 -51.863826   0.005871   
1.967600e-01 19    -51.937664   0.008345 -51.929319 -51.863760   0.007806   
2.759970e-01 20    -51.937519   0.010723 -51.926796 -51.863586   0.010124   
3.871000e-01 21    -51.937167   0.013617 -51.923549 -51.863195   0.012888   
5.431000e-01 22    -51.936343   0.017145 -51.919198 -51.862322   0.016273   
7.617000e-01 23    -51.934952   0.020993 -51.913959 -51.860858   0.020137   
1.068500e+00 24    -51.933296   0.024665 -51.908631 -51.859104   0.024091   
1.498800e+00 25    -51.931528   0.028270 -51.903258 -51.857216   0.028179   
2.102400e+00 26    -51.929651   0.032164 -51.897487 -51.855197   0.032562   
2.949000e+00 27    -51.927705   0.036712 -51.890994 -51.853098   0.037365   
4.136600e+00 28    -51.925877   0.042354 -51.883523 -51.851104   0.042945   
5.802500e+00 29    -51.924316   0.049894 -51.874422 -51.849372   0.050125   
8.139200e+00 30    -51.922903   0.060600 -51.862303 -51.847769   0.060295   
1.141700e+01 31    -51.921638   0.075725 -51.845913 -51.846299   0.074412   
1.601470e+01 32    -51.920976   0.096664 -51.824311 -51.845450   0.093392   
2.246400e+01 33    -51.922076   0.125679 -51.796397 -51.846461   0.119716   
3.151050e+01 34    -51.925639   0.168002 -51.757636 -51.850044   0.159742   
4.420010e+01 35    -51.931075   0.232062 -51.699014 -51.855489   0.221176   
6.200000e+01 36    -51.938309   0.327087 -51.611222 -51.862724   0.311013   
8.577500e+01 37    -51.946660   0.458926 -51.487734 -51.871098   0.435665   
1.095500e+02 38    -51.953364   0.593454 -51.359910 -51.877826   0.566979   
1.333250e+02 39    -51.959786   0.728288 -51.231498 -51.884328   0.703817   
1.571000e+02 40    -51.970148   0.907583 -51.062565 -51.894863   0.891160   
1.808750e+02 41    -51.985964   1.127989 -50.857975 -51.910935   1.123980   
2.046500e+02 42    -52.009398   1.388532 -50.620867 -51.934716   1.395820   
2.284250e+02 43    -52.041307   1.670405 -50.370902 -51.967300   1.683070   
2.522000e+02 44    -52.084252   1.970049 -50.114203 -52.011453   1.979908   
2.759750e+02 45    -52.144331   2.290807 -49.853524 -52.073996   2.290162   
2.997500e+02 46    -52.233897   2.637608 -49.596289 -52.168397   2.620119   
3.235250e+02 47    -52.355823   3.036481 -49.319342 -52.297416   2.996786   
3.473000e+02 48    -52.495269   3.524968 -48.970301 -52.444828   3.460422   
3.710750e+02 49    -52

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')